# 4. surprise이용 추천시스템

## 2. 실제 영화 평점 데이터로 surprise 패키지 사용해보기

In [250]:
from surprise import SVD      # 행렬 분해 알고리즘 
from surprise import Dataset  # 내장 데이터 불러오는 라이브러리
from surprise import accuracy # rmse로 예측 에러 평가
from surprise.model_selection import train_test_split  # 학습셋, 테스트셋 분리

### 4.1 Cross Validation(교차 검증)과 GridSearchCV(하이퍼 파라미터 튜닝)

#### Cross Validation(교차 검증)

In [275]:
from surprise import SVD      # MF(행렬 분해) 알고리즘 
from surprise import Dataset  # 내장 데이터 불러오는 라이브러리
from surprise import accuracy # accuracy의 rmse메소드로 예측 에러 평가
from surprise.model_selection import train_test_split  # 학습셋, 테스트셋 분리
from surprise.model_selection import cross_validate

In [276]:
%%time
import pandas as pd
from surprise import Reader, Dataset

ratings = pd.read_excel("iherb_ratings.xlsx")
ratings = ratings.drop("Unnamed: 0", axis=1)  # 불필요한 칼럼 삭제

print(ratings.shape)
ratings.head()

(36649, 3)
Wall time: 3.56 s


,userid,item,rating
0,0,"Sierra Fit,프로틴 크리스프, 솔티드 토피, 12개입, 각 56g(1.98oz)",5
1,0,"Sierra Fit,프로틴 크리스프, 땅콩버터, 바 12개입, 각 56g(1.98oz)",5
2,0,"Sierra Fit,프로틴 크리스프, 초콜릿 칩 쿠키 도우, 12개입, 각 56g(...",3
3,0,"Sierra Fit,유청 단백질 컴플리트, 바닐라, 2.27kg(5lb)",5
4,0,"Bob's Red Mill,Organic Quick Cooking Rolled Oa...",5


In [277]:
# 데이터 로드하기
reader = Reader(rating_scale=(0.5, 5.0))
# ratings DataFrame 에서 컬럼은 userId, itemId, rating 순서를 지켜야 합니다. 
data = Dataset.load_from_df(ratings[['userid', 'item', 'rating']], reader)

# 데이터를 학습셋, 테스트셋으로 나누기
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

# 행렬 분해 알고리즘 학습 및 예측, 평가(RMSE)
algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset) 
predictions = algo.test( testset )
accuracy.rmse(predictions)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True) 

RMSE: 0.5445
Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.5673  0.5620  0.5690  0.5331  0.5761  0.5615  0.0149  
MAE (testset)     0.3276  0.3283  0.3263  0.3149  0.3297  0.3254  0.0053  
Fit time          1.50    1.85    1.70    1.59    1.49    1.63    0.14    
Test time         0.06    0.08    0.06    0.06    0.06    0.06    0.01    


{'test_rmse': array([0.56728035, 0.56195026, 0.56897878, 0.53307086, 0.57606513]),
 'test_mae': array([0.32762717, 0.32827168, 0.32631811, 0.31490271, 0.32969704]),
 'fit_time': (1.5009839534759521,
  1.8510513305664062,
  1.7044384479522705,
  1.5897481441497803,
  1.494006872177124),
 'test_time': (0.06386613845825195,
  0.07579421997070312,
  0.0608372688293457,
  0.06482601165771484,
  0.05884146690368652)}

#### GridSearchCV(하이퍼 파라미터 튜닝)

In [278]:
from surprise.model_selection import GridSearchCV

# 최적화할 파라미터들을 딕셔너리 형태로 지정. 
param_grid = {'n_epochs': [20, 40, 60], 'n_factors': [50, 100, 200] }

# GridSearchCV 세팅 : CV를 3개 폴드 세트로 지정, 성능 평가는 rmse, mse 로 수행 하도록 GridSearchCV 구성
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs

In [279]:
%%time
# GridSearchCV로 학습
gs.fit(data)

# 최고 RMSE Evaluation 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

# 20만 건 데이터 기준 실행시간 (원본 데이터 2천만 건이면 8분~10분 정도 걸림)

0.5623133480159631
{'n_epochs': 20, 'n_factors': 50}
Wall time: 2min 12s


## 4. Surprise 를 이용한 개인화 영화 추천 시스템 구축

### 전체 데이터로 학습 진행(학습셋, 테스트셋 나누지 않고)

In [280]:
%%time
from surprise.dataset import DatasetAutoFolds  # 데이터 세트 전체를 학습 데이터로 사용할 수 있게 해주는 라이브러리

reader = Reader(line_format='user item rating', sep=',', rating_scale=(0.5, 5))
# DatasetAutoFolds 클래스를 ratings_noh.csv 파일 기반으로 생성. 
data_folds = DatasetAutoFolds.load_from_df(ratings[['userid', 'item', 'rating']], reader)

# 전체 데이터를 학습데이터로 생성함.
trainset = data_folds.build_full_trainset()
# 실행시간 3분

Wall time: 138 ms


In [281]:
%%time
# SVD 협업필터링으로 추천모델 학습(하이퍼 파라미터는 앞서 그리드서치로 구한 것들)
algo = SVD(n_epochs=20, n_factors=50, random_state=0)

algo.fit(trainset)
# 실행시간 11분

Wall time: 1.9 s


In [282]:
# 영화에 대한 상세 속성 정보 DataFrame로딩
items = pd.read_excel('iherb_ratings.xlsx')

# userId=9 의 movieId 데이터 추출하여 movieId=42 데이터가 있는지 확인. 
itemIds = ratings[ratings['userId']==9]['item']
if itemIds.count() == 0:
    print('사용자 아이디 9는==4 영화 아이디 42의 평점 없음')

print(item[item['item']==42])

KeyError: 'userId'

In [79]:
# predict 메소드를 사용해서 예측 평점 구하기
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 2.95   {'was_impossible': False}


### 특정 사용자가 아직 안 본 전체 영화 추출 후 예측 평점 순으로 영화 추천

In [80]:
def get_unseen_surprise(ratings, movies, userId):
    #입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
    seen_movies = ratings[ratings['userId']== userId]['movieId'].tolist()
    
    # 모든 영화들의 movieId를 리스트로 생성. 
    total_movies = movies['movieId'].tolist()
    
    # 모든 영화들의 movieId중 이미 평점을 매긴 영화의 movieId를 제외하여 리스트로 생성
    unseen_movies= [movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화수:',len(seen_movies), '추천대상 영화수:',len(unseen_movies), \
          '전체 영화수:',len(total_movies))
    
    return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화수: 1 추천대상 영화수: 27277 전체 영화수: 27278


In [81]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
    # 알고리즘 객체의 predict() 메서드를 평점이 없는 영화(27243개)에 반복 수행한 후 결과를 list 객체로 저장
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]
    
    # predictions list 객체는 surprise의 Predictions 객체를 원소로 가지고 있음.
    # [Prediction(uid='9', iid='1', est=3.69), Prediction(uid='9', iid='2', est=2.98),,,,]
    # 이를 est 값으로 정렬하기 위해서 아래의 sortkey_est 함수를 정의함.
    # sortkey_est 함수는 list 객체의 sort() 함수의 키 값으로 사용되어 정렬 수행.
    def sortkey_est(pred):
        return pred.est
    
    # sortkey_est( ) 반환값의 내림 차순으로 정렬 수행하고 top_n개의 최상위 값 추출.
    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions= predictions[:top_n]
    
    # top_n으로 추출된 영화의 정보 추출. 영화 아이디, 추천 예상 평점, 제목 추출
    top_movie_ids = [ int(pred.iid) for pred in top_predictions]
    top_movie_rating = [ pred.est for pred in top_predictions]
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']
    top_movie_preds = [ (id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]
    
    return top_movie_preds

In [82]:
unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)
print("")
print('##### Top-10 추천 영화 리스트 #####')

for top_movie in top_movie_preds:
    print(top_movie[1], ":", top_movie[2])

평점 매긴 영화수: 1 추천대상 영화수: 27277 전체 영화수: 27278

##### Top-10 추천 영화 리스트 #####
Usual Suspects, The (1995) : 4.403255960567584
Braveheart (1995) : 4.354730709188801
Pulp Fiction (1994) : 4.279467874657644
Shawshank Redemption, The (1994) : 4.2719954041396715
Schindler's List (1993) : 4.268033622651485
Silence of the Lambs, The (1991) : 4.218225009929055
Godfather, The (1972) : 4.216492633386306
Lord of the Rings: The Return of the King, The (2003) : 4.206242413033943
Personal Journey with Martin Scorsese Through American Movies, A (1995) : 4.204346268855151
Frozen Planet (2011) : 4.197629744293488


In [ ]:
## 4. Surprise 를 이용한 개인화 영화 추천 시스템 구축

특정 사용자(userId=9)가 아직 보지 않은 영화 중에서 추천해주는 추천시스템을 만들어보자

### 전체 데이터로 학습 진행(학습셋, 테스트셋 나누지 않고)

%%time
from surprise.dataset import DatasetAutoFolds  # 데이터 세트 전체를 학습 데이터로 사용할 수 있게 해주는 라이브러리

reader = Reader(line_format='user item rating', sep=',', rating_scale=(0.5, 5))
# DatasetAutoFolds 클래스를 ratings_noh.csv 파일 기반으로 생성. 
data_folds = DatasetAutoFolds.load_from_df(ratings[['userid', 'item', 'rating']], reader)

# 전체 데이터를 학습데이터로 생성함.
trainset = data_folds.build_full_trainset()
# 실행시간 3분

%%time
# SVD 협업필터링으로 추천모델 학습(하이퍼 파라미터는 앞서 그리드서치로 구한 것들)
algo = SVD(n_epochs=20, n_factors=50, random_state=0)

algo.fit(trainset)
# 실행시간 11분

특정 사용자 userId=9 가 아직 안 본 영화 movieId=42 예측 평점은?

# 영화에 대한 상세 속성 정보 DataFrame로딩
items = pd.read_excel('iherb_ratings.xlsx')

# userId=9 의 movieId 데이터 추출하여 movieId=42 데이터가 있는지 확인. 
itemIds = ratings[ratings['userId']==9]['item']
if itemIds.count() == 0:
    print('사용자 아이디 9는==4 영화 아이디 42의 평점 없음')

print(item[item['item']==42])

# predict 메소드를 사용해서 예측 평점 구하기
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose=True)

-> 9번 사용자의 42번 영화에 대한 예측 평점은 2.95점이다.

### 특정 사용자가 아직 안 본 전체 영화 추출 후 예측 평점 순으로 영화 추천

def get_unseen_surprise(ratings, movies, userId):
    #입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
    seen_movies = ratings[ratings['userId']== userId]['movieId'].tolist()
    
    # 모든 영화들의 movieId를 리스트로 생성. 
    total_movies = movies['movieId'].tolist()
    
    # 모든 영화들의 movieId중 이미 평점을 매긴 영화의 movieId를 제외하여 리스트로 생성
    unseen_movies= [movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화수:',len(seen_movies), '추천대상 영화수:',len(unseen_movies), \
          '전체 영화수:',len(total_movies))
    
    return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)

def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
    # 알고리즘 객체의 predict() 메서드를 평점이 없는 영화(27243개)에 반복 수행한 후 결과를 list 객체로 저장
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]
    
    # predictions list 객체는 surprise의 Predictions 객체를 원소로 가지고 있음.
    # [Prediction(uid='9', iid='1', est=3.69), Prediction(uid='9', iid='2', est=2.98),,,,]
    # 이를 est 값으로 정렬하기 위해서 아래의 sortkey_est 함수를 정의함.
    # sortkey_est 함수는 list 객체의 sort() 함수의 키 값으로 사용되어 정렬 수행.
    def sortkey_est(pred):
        return pred.est
    
    # sortkey_est( ) 반환값의 내림 차순으로 정렬 수행하고 top_n개의 최상위 값 추출.
    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions= predictions[:top_n]
    
    # top_n으로 추출된 영화의 정보 추출. 영화 아이디, 추천 예상 평점, 제목 추출
    top_movie_ids = [ int(pred.iid) for pred in top_predictions]
    top_movie_rating = [ pred.est for pred in top_predictions]
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']
    top_movie_preds = [ (id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]
    
    return top_movie_preds

unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)
print("")
print('##### Top-10 추천 영화 리스트 #####')

for top_movie in top_movie_preds:
    print(top_movie[1], ":", top_movie[2])

-> 사용자 9번에게는 주로 서스펜스/스릴러/범죄/액션 영화들이 추천되었다.

결론 : 
이렇게 userId, itemId, rating 데이터들을 가지고, 
surprise 패키지를 이용하면 
쉽게 개인화 추천시스템을 구축할 수 있다.